In [1]:
%load_ext autoreload
%autoreload 2

from trading import api

trades_unit = api.fetch_data()
trades_date = api.convert_to_time(trades_unit)
trades_unit = api.compute_stats(trades_unit)
trades_date = api.compute_stats(trades_date.reset_index(), all_stats=False).set_index(('Date', '.'))

# print(trades_unit.columns)
# print(trades_unit.head())
# print(trades_date.columns)
# print(trades_date.head())

In [172]:
import plotly.graph_objects as go
trades = trades_unit.copy()
periods = ['Globally', 'Yearly', 'Monthly', 'Weekly']

fig = go.Figure(go.Bar(x=trades.index.to_list(), y=trades[('Gain', 'Globally')].to_list(), name='Trades'))

for col in trades['Balance'].columns.get_level_values(0):
    for period in periods:
        if period in col:
            fig.add_trace(go.Scatter(x=trades.index.to_list(), y=trades[('Balance', col)].to_list(), name='Balance '+col.replace('_', ' ')))

fig.show()

In [3]:
import pandas as pd
import numpy as np
trades = trades_unit.copy()
PERIOD = ['Globally', 'Yearly', 'Monthly', 'Weekly']
s = trades.ffill().iloc[-1]

stats = pd.DataFrame()
for metric, period in s.index:
    for p in ['Globally', 'Yearly', 'Monthly', 'Weekly']:
        if p in period and metric in ['Count', 'Balance', 'CountTP', 'RateTP', 'PayoffTP', 'CountBE', 'RateBE', 'PayoffBE', 'CountSL', 'RateSL', 'PayoffSL']:
            stats.loc[period, metric] = s[(metric, period)]
stats['WinRate'] = stats['RateTP']
stats['PayoffRatio'] = stats['PayoffTP'] / -stats['PayoffSL']
stats

,Balance,Count,CountTP,RateTP,PayoffTP,CountBE,RateBE,PayoffBE,CountSL,RateSL,PayoffSL,WinRate,PayoffRatio
Globally,-237.35,131.0,47.0,0.364341,32.497021,17.0,0.133858,0.566471,67.0,0.511450,-26.482687,0.364341,1.227104
Yearly_2021,296.71,71.0,28.0,0.394366,41.378929,9.0,0.180000,1.737778,34.0,0.485714,-25.810000,0.394366,1.603213
Monthly_2021-8,294.59,2.0,2.0,1.000000,147.295000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
Weekly_2021-34,35.09,1.0,1.0,1.000000,35.090000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
Weekly_2021-35,267.32,3.0,1.0,1.000000,259.500000,2.0,0.666667,3.910000,NaN,NaN,NaN,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Weekly_2023-4,-13.34,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,-13.340000,NaN,NaN
Monthly_2023-2,23.40,2.0,1.0,1.000000,46.800000,NaN,NaN,NaN,1.0,0.500000,-23.400000,1.000000,2.000000
Weekly_2023-6,23.40,2.0,1.0,1.000000,46.800000,NaN,NaN,NaN,1.0,0.500000,-23.400000,1.000000,2.000000
Monthly_2023-3,-24.22,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,-24.220000,NaN,NaN


In [125]:
gain = [col for col in trades.columns if 'Gain' in col] 
balance = [col for col in trades.columns if 'Balance' in col] 
trades_date = pd.concat([
    pd.DataFrame({'Date': pd.date_range(start=trades['Date'].min(), end=trades['Date'].max())}).set_index('Date'),
    trades.set_index('Date').groupby('Date')[gain].sum(),
    trades.set_index('Date').groupby('Date')[balance].last(),
], axis=1)
trades_date[[col for col in trades.columns if 'Balance_' in col]].head(50)

,Balance_Globally,Balance_Yearly_2021,Balance_Monthly_2021-8,Balance_Weekly_2021-34,Balance_Weekly_2021-35,Balance_Monthly_2021-9,Balance_Weekly_2021-36,Balance_Weekly_2021-37,Balance_Weekly_2021-38,Balance_Weekly_2021-39,...,Balance_Weekly_2022-50,Balance_Yearly_2023,Balance_Monthly_2023-1,Balance_Weekly_2023-2,Balance_Weekly_2023-3,Balance_Weekly_2023-4,Balance_Monthly_2023-2,Balance_Weekly_2023-6,Balance_Monthly_2023-3,Balance_Weekly_2023-13
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-27,35.09,35.09,35.09,35.09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-31,294.59,294.59,294.59,NaN,259.50,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-02,298.91,298.91,NaN,NaN,263.82,4.32,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-03,302.41,302.41,NaN,NaN,267.32,7.82,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df = pd.DataFrame()
df.shape == (0,0)

True